<a href="https://colab.research.google.com/github/Manish533005/nlp_chatbot/blob/main/training__data__building__model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import numpy as np
import json
import spacy
import random
import pickle
print(tf.__version__)

1.15.2


In [49]:
#loading dataset(json file) into a file 
data_file=open("chatbot_dataset.json","r")
intents=json.load(data_file)

In [50]:
words=[] #contains the total words in the dataset
classes=[]# contains the categories
documents=[]#contains patterns and categories as tuples in the list
ignore_words=["?",".","!"," ",","]
en_lang_model=spacy.load("en_core_web_sm")

all_stopwords=en_lang_model.Defaults.stop_words
print(all_stopwords)


{'own', 'next', 'hereupon', 'ten', 'a', 'almost', 'i', 'had', 'twelve', 'mostly', 'without', 'me', 'off', 'behind', 'in', 'before', 'hereafter', 'his', 'against', 'get', 'into', 'more', 'least', 'up', 'except', '’m', 'are', 'forty', 'although', 'were', 'doing', 'become', 'because', 'enough', 'nine', 'third', 'she', 'namely', 'either', 'whereby', 'very', 'call', 'who', 'fifty', 'amount', 'here', 'each', 'somewhere', '‘d', 'about', 'may', 'our', 'per', 'thereafter', 'throughout', 'where', 'whereupon', 'at', 'within', 'made', 'empty', 'nor', 'take', 'twenty', 'whither', 'put', 'then', 'would', 'any', 'over', 'there', 'he', "'re", 'due', 'its', '’re', 'above', 'moreover', 'perhaps', 'but', 'or', 'both', 'rather', 'hers', 'anything', 'cannot', 'always', 'hundred', 'via', 'as', 'was', 'whatever', 'done', 'thereupon', 'bottom', 'none', 'with', 'an', 'part', 'used', 'be', 'should', 'fifteen', 'top', 'sometime', 'well', 'ourselves', 'has', 'again', 'neither', "'m", 'him', 'latterly', 'once', 's

In [51]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        tokens= en_lang_model(pattern)
        l=[]
        for i in tokens:
             i=str(i)
             l.append(i)
        words.extend(l)
        #add documents in the corpus
        
        documents.append((l, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [52]:
print(words)
print(documents)
print(classes)

['Hello', 'Hi', 'Hey', 'Hi', 'there', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'order', 'food', 'food', 'order', 'order', 'I', 'want', 'to', 'order', 'food', 'What', 'variety', 'of', 'foods', 'are', 'available', 'Is', 'food', 'available', 'Can', 'I', 'order', 'pizza', 'Are', 'veg', 'pizzas', 'available', 'Are', 'non', 'veg', 'pizzas', 'available', 'I', 'am', 'hungry', 'I', 'want', 'some', 'food', 'is', 'food', 'available', 'Can', 'I', 'view', 'the', 'menu', 'please', 'Menu', 'Menu', 'please', 'What', 'is', 'the', 'menu', 'What', 'can', 'i', 'order', 'Is', 'combo', 'available', 'Combo', 'please', 'Combo', 'What', 'is', 'today', "'s", 'menu', 'What', 'food', 'is', 'there', 'today', 'Is', 'food', 'there', 'Can', 'you', 'show', 'menu', 'Can', 'I', 'see', 'the', 'menu', 'What', 'have', 'you', 'got', 'What', 'can', 'I', 'get', 'Show', 'menu', 'Show', 'what', 'I', 'can', 'get', 'Menu', 'menu', 'I', 'am', 'hungry', 'cancel', 

In [53]:
#prepeocessing of the dataset
#lemmatize each word(lower)
#remove the duplicate words
# neglecting the words (ingnore_words)
words=[en_lang_model(w.lower())[0].lemma_  for w in words if w not in ignore_words]
words=[ w for w in words if w not in all_stopwords]
#sorting the words in alphabetical order
words=sorted(list(set(words)))
#sorting the categories in alphabetical order
classes = sorted(list(set(classes)))

In [54]:
print(words)

['-', '-PRON-', 'add', 'available', 'beverage', 'bye', 'cancel', 'cancle', 'change', 'chat', 'chicken', 'combo', 'confirm', 'cool', 'dominator', 'drink', 'edit', 'food', 'goodbye', 'hard', 'hello', 'hey', 'hi', 'hungry', 'item', 'large', 'later', 'medium', 'menu', 'nice', 'non', 'nonveg', 'order', 'pizza', 'remove', 'request', 'slide', 'small', 'till', 'time', 'today', 'variety', 'veg', 'view', 'want']


In [55]:
print(classes)

['Beverages', 'Bye', 'Non-Veg Pizza', 'Slides', 'Veg Pizza', 'add_menu', 'cancel_food', 'confirm_order', 'edit_order', 'food_item', 'greetings', 'order_food', 'pizza_size', 'remove_menu']


In [56]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [57]:
#creating the training data
#encoding technique
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [en_lang_model(word.lower())[0].lemma_ for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
       
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
model =Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_6.h5', hist)

print("model created")

In [59]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 128)               5888      
_________________________________________________________________
dropout_24 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_26 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 14)              